# Eat Safe, Love

## Notebook Set Up

In [30]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [31]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [32]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [33]:
# review the collections in our database
pprint(db.list_collection_names())

['establishments']


In [34]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [35]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(f"Documents with hygiene score of 20: {count}")

'Documents with hygiene score of 20: 0'


In [36]:
# Convert the result to a Pandas DataFrame
hygiene = establishments.find(query)
hygiene_df = pd.DataFrame(hygiene)

# Display the number of rows in the DataFrame
print(len(hygiene_df))

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [37]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"$and": [{"AddressLine1": {"$regex": "London"}}, {"RatingValue": {"$gte": 4}}]}

# Use count_documents to display the number of documents in the result
london_count = establishments.count_documents(query)
print(f"Number of establishments in London with a rating value of 4 or higher: {london_count}")

# Display the first document in the results using pprint
london = establishments.find(query)
pprint(next(london, "No documents found"))


Number of establishments in London with a rating value of 4 or higher: 0
'No documents found'


In [38]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {hygiene_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(hygiene_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [39]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074
longitude = -0.1278

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)  

In [40]:
# Convert result to Pandas DataFrame
results_df = pd.DataFrame(list(establishments.find(query).sort(sort).limit(limit)))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [41]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# 4. Limits the results to 10 documents
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]   

# Print the number of documents in the result
hygiene_count = establishments.aggregate(pipeline)
print(f"Number of establishments with a hygiene score of 0: {hygiene_count}")   
# Print the first 10 results
for result in hygiene_count:
    pprint(result)  


Number of establishments with a hygiene score of 0: <pymongo.command_cursor.CommandCursor object at 0x0000021625FB2020>


In [42]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list(establishments.find(query)))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {hygiene_df.shape[0]}")
# Display the first 10 rows of the DataFrame
print(hygiene_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []
